In [1]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess


In [4]:
main=pd.read_csv('../data/train.csv')
sub=pd.read_csv('../data/store.csv')
data=pd.merge(main,sub,on='Store',how='left')
data['Date']=pd.to_datetime(data['Date'])
data.sort_values(by=['Store','Date'],inplace=True)
data.set_index('Date',inplace=True)
data.head()

/tmp/ipykernel_38841/1517388885.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  main=pd.read_csv('../data/train.csv')


,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
Date,,,,,,,,,,,,,,,,,
2013-01-01,1,2,0,0,0,0,a,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
2013-01-02,1,3,5530,668,1,0,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
2013-01-03,1,4,4327,578,1,0,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
2013-01-04,1,5,4486,619,1,0,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
2013-01-05,1,6,4997,635,1,0,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN


In [37]:
def make_lag(data,lags,week):
    for lag in range(1,lags+1):
            data['lag_{}_{}'.format(lag,week)]=data['Sales'].shift(lag+7*week)
    return data
temp=pd.DataFrame()
temp=data[['Store','Sales']].copy()
print(temp.columns)
make_lag(temp,3,0)
make_lag(temp,1,1)
temp.dropna(inplace=True)
temp['rolling_mean_7']=(
    temp['Sales'].shift(1).rolling(
    windows=7,
    center=False,
    min_periods=1
 ).mean()
)
temp['rolling_std_7']=(
    temp['Sales'].shift(1).rolling(
    windows=7,
    center=False,
    min_periods=1
 ).std()
)

Index(['Store', 'Sales'], dtype='object')


TypeError: NDFrame.rolling() got an unexpected keyword argument 'windows'

In [11]:
data.head(1)

,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
Date,,,,,,,,,,,,,,,,,
2013-01-01,1,2,0,0,0,0,a,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN


In [28]:
baseline_features=data.copy()
baseline_features.drop(['Sales','Customers','CompetitionDistance',	'CompetitionOpenSinceMonth',
                        	'CompetitionOpenSinceYear',	'Promo2','Promo2SinceWeek','Promo2SinceYear','PromoInterval'],axis=1,inplace=True)
baseline_features['day']=((data.index - data.index.min())).days
to_category=['Open','Promo','StateHoliday','SchoolHoliday','StoreType','Assortment']
baseline_features[to_category]=baseline_features[to_category].astype('category')
baseline_features['DayofMonth']=data.index.day
baseline_features.head(10)

,Store,DayOfWeek,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,day,DayofMonth
Date,,,,,,,,,,
2013-01-01,1,2,0,0,a,1,c,a,0,1
2013-01-02,1,3,1,0,0,1,c,a,1,2
2013-01-03,1,4,1,0,0,1,c,a,2,3
2013-01-04,1,5,1,0,0,1,c,a,3,4
2013-01-05,1,6,1,0,0,1,c,a,4,5
2013-01-06,1,7,0,0,0,1,c,a,5,6
2013-01-07,1,1,1,1,0,1,c,a,6,7
2013-01-08,1,2,1,1,0,1,c,a,7,8
2013-01-09,1,3,1,1,0,1,c,a,8,9


In [35]:
res=[]
for Store,df_store in data.groupby('Store'):
    fouriers = CalendarFourier(freq="A", order=10)#生成日历傅里叶特征
    dp = DeterministicProcess(
        index=df_store.index,
        constant=True,               #是否包含常数项
        order=1,                     #线性趋势
        seasonal=False,               #是否包含季节性虚拟变量
        additional_terms=[fouriers], #添加傅里叶特征
        drop=True,           
    )
    X=dp.in_sample()  #生成样本内的确定性特征
    X['Store']=Store
    X['Date']=df_store.index
    res.append(X)
X_dp = pd.concat(res, ignore_index=True)
baseline_features=baseline_features.merge(X_dp,on=['Store','Date'],how='left')
y = data["Sales"]               #目标变量
baseline_features.head(10)


/home/yjjjc/python/myv1/lib/python3.12/site-packages/statsmodels/tsa/deterministic.py:569: FutureWarning: 'A' is deprecated and will be removed in a future version, please use 'YE' instead.
  index = pd.date_range("2020-01-01", freq=freq, periods=1)
/home/yjjjc/python/myv1/lib/python3.12/site-packages/statsmodels/tsa/deterministic.py:569: FutureWarning: 'A' is deprecated and will be removed in a future version, please use 'YE' instead.
  index = pd.date_range("2020-01-01", freq=freq, periods=1)
/home/yjjjc/python/myv1/lib/python3.12/site-packages/statsmodels/tsa/deterministic.py:569: FutureWarning: 'A' is deprecated and will be removed in a future version, please use 'YE' instead.
  index = pd.date_range("2020-01-01", freq=freq, periods=1)
/home/yjjjc/python/myv1/lib/python3.12/site-packages/statsmodels/tsa/deterministic.py:569: FutureWarning: 'A' is deprecated and will be removed in a future version, please use 'YE' instead.
  index = pd.date_range("2020-01-01", freq=freq, periods=1)


,Store,Date,DayOfWeek,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,day,...,"sin(6,freq=YE-DEC)","cos(6,freq=YE-DEC)","sin(7,freq=YE-DEC)","cos(7,freq=YE-DEC)","sin(8,freq=YE-DEC)","cos(8,freq=YE-DEC)","sin(9,freq=YE-DEC)","cos(9,freq=YE-DEC)","sin(10,freq=YE-DEC)","cos(10,freq=YE-DEC)"
0,1,2013-01-01,2,0,0,a,1,c,a,0,...,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000
1,1,2013-01-02,3,1,0,0,1,c,a,1,...,0.103102,0.994671,0.120208,0.992749,0.137279,0.990532,0.154309,0.988023,0.171293,0.985220
2,1,2013-01-03,4,1,0,0,1,c,a,2,...,0.205104,0.978740,0.238673,0.971100,0.271958,0.962309,0.304921,0.952378,0.337523,0.941317
3,1,2013-01-04,5,1,0,0,1,c,a,3,...,0.304921,0.952378,0.353676,0.935368,0.401488,0.915864,0.448229,0.893919,0.493776,0.869589
4,1,2013-01-05,6,1,0,0,1,c,a,4,...,0.401488,0.915864,0.463550,0.886071,0.523416,0.852078,0.580800,0.814046,0.635432,0.772157
5,1,2013-01-06,7,0,0,0,1,c,a,5,...,0.493776,0.869589,0.566702,0.823923,0.635432,0.772157,0.699458,0.714673,0.758306,0.651899
6,1,2013-01-07,1,1,1,0,1,c,a,6,...,0.580800,0.814046,0.661635,0.749826,0.735417,0.677615,0.801361,0.598181,0.858764,0.512371
7,1,2013-01-08,2,1,1,0,1,c,a,7,...,0.661635,0.749826,0.746972,0.664855,0.821477,0.570242,0.884068,0.467359,0.933837,0.357698
8,1,2013-01-09,3,1,1,0,1,c,a,8,...,0.735417,0.677615,0.821477,0.570242,0.891981,0.452072,0.945596,0.325342,0.981306,0.192452
9,1,2013-01-10,4,1,1,0,1,c,a,9,...,0.801361,0.598181,0.884068,0.467359,0.945596,0.325342,0.984474,0.175531,0.999769,0.021516
